<a href="https://colab.research.google.com/github/xHexlabx/SuperAI_SS4_Recap/blob/main/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/Liver_Ultrasound_Detection_Fix_Predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SuperAI Season 4 - Level 2 Hackathon - Liver_Ultrasound_Detection_Fix_Predictions

## Mounting Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Load Datasets

In [ ]:
!unzip /content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/zips/liver-ultrasound-detection.zip

Streaming output truncated to the last 5000 lines.
  inflating: val/val/annotations/95431.txt  
  inflating: val/val/annotations/95442.txt  
  inflating: val/val/annotations/95505.txt  
  inflating: val/val/annotations/95579.txt  
  inflating: val/val/annotations/956.txt  
  inflating: val/val/annotations/95620.txt  
  inflating: val/val/annotations/95632.txt  
  inflating: val/val/annotations/95685.txt  
  inflating: val/val/annotations/95697.txt  
  inflating: val/val/annotations/9571.txt  
  inflating: val/val/annotations/95729.txt  
  inflating: val/val/annotations/9573.txt  
  inflating: val/val/annotations/9576.txt  
  inflating: val/val/annotations/95785.txt  
  inflating: val/val/annotations/95827.txt  
  inflating: val/val/annotations/95851.txt  
  inflating: val/val/annotations/95897.txt  
  inflating: val/val/annotations/96013.txt  
  inflating: val/val/annotations/96048.txt  
  inflating: val/val/annotations/96208.txt  
  inflating: val/val/annotations/96316.txt  
  inflati

## Fix Predictions

In [ ]:
!pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 11.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
submission_df = pd.read_csv('/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/submissions/last_submission_1.csv')
dummy_df = pd.read_csv('/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/submissions/last_submission_1.csv')

In [ ]:
submission_df

,Image File,Annotation,Label
0,18999,"[[210.75, 245.33333333333334, 254.25, 354.6666...",[1]
1,102037,"[[197.25, 228.0, 269.25, 350.66666666666663]]",[4]
2,138340,"[[63.75, 313.33333333333326, 133.5, 444.0]]",[4]
3,140589,"[[594.9697265625, 378.3086395263672, 660.30224...",[4]
4,20235,"[[294.8218994140625, 114.39697647094727, 337.5...",[1]
...,...,...,...
5148,3575,[],[]
5149,190,[],[]
5150,1404,"[[1256.34619140625, 896.2747497558594, 1491.74...",[3]
5151,184,[],[]


In [ ]:
test_image_path = '/content/test/test/images'

In [ ]:
import torch

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import torch
import timm
from PIL import Image
from tqdm.notebook import tqdm
from torchvision import transforms
import numpy as np

model = timm.create_model("timm/maxvit_tiny_tf_224.in1k", checkpoint_path='/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/datasets/model_checkpoints/maxvit_model_67.pth' , pretrained = False , num_classes = 2)
model.to(device)
# Set the model to evaluation mode
model.eval()

transform = transforms.Compose([
    transforms.Resize((224 , 224)),  # Resize the image to 128x128y
    transforms.ToTensor(),  # Convert the image to a PyTorch tensor
])

cnt = 0

for idx in tqdm(range(len(submission_df))) :

    row = submission_df.iloc[idx]
    image_name = row['Image File']

    image = Image.open(f'{test_image_path}/{image_name}.jpg')
    image = image.convert('RGB')

    image = transform(image).unsqueeze(0).to(device)

    output = model(image)

    _ , predicted = torch.max(output.data, 1)

    if predicted == 0 :

        cnt += 1

        dummy_df.iloc[idx]['Annotation'] = []
        dummy_df.iloc[idx]['Label'] = []
# The model is now ready to use


  0%|          | 0/5153 [00:00<?, ?it/s]

<ipython-input-18-574ca98e138a>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dummy_df.iloc[idx]['Annotation'] = []
<ipython-input-18-574ca98e138a>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dummy_df.iloc[idx]['Label'] = []


In [ ]:
print('change' , cnt / len(submission_df) * 100 , '%')

change 51.13526101300213 %


In [ ]:
dummy_df.to_csv('/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/submissions/enhance_0189.csv' ,index = False)

## Ensemble

In [ ]:
import pandas as pd
import ast

In [ ]:
sub_1 = pd.read_csv('/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/submissions/sub_1.csv')

In [ ]:
def combine(sub_1 , sub_2):

    size = len(sub_1)

    sub_1['Annotation'] = sub_1['Annotation'].apply(lambda x: ast.literal_eval(x) if "[" in x else x)
    sub_1['Label'] = sub_1['Label'].apply(lambda x: ast.literal_eval(x) if "[" in x else x)


    sub_2['Annotation'] = sub_2['Annotation'].apply(lambda x: ast.literal_eval(x) if "[" in x else x)
    sub_2['Label'] = sub_2['Label'].apply(lambda x: ast.literal_eval(x) if "[" in x else x)

    for idx in range(3 , size) :

        for pos in sub_2.iloc[idx]['Annotation'] :

            sub_1.iloc[idx]['Annotation'].append(pos)

        for label in sub_2.iloc[idx]['Label'] :

            sub_1.iloc[idx]['Label'].append(label)

    return sub_1

In [ ]:
for i in range(2 , 8) :

    sub = pd.read_csv(f'/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/submissions/sub_{i}.csv')
    sub_1 = combine(sub_1 , sub)

In [ ]:
sub_1.head(50)

,Image File,Annotation,Label
0,18999,"[[210.75, 245.33333333333334, 254.25, 354.6666...",[1]
1,102037,"[[197.25, 228.0, 269.25, 350.66666666666663]]",[4]
2,138340,"[[63.75, 313.33333333333326, 133.5, 444.0]]",[4]
3,140589,"[[598.7199401855469, 376.12420654296875, 662.1...","[0, 4]"
4,20235,"[[292.3599395751953, 115.42367935180664, 337.8...",[1]
5,68271,"[[487.86924743652344, 336.2770690917969, 535.0...",[4]
6,102209,"[[459.3815002441406, 308.42144775390625, 509.7...","[1, 3]"
7,77806,"[[474.2916259765625, 167.37203979492188, 645.9...","[2, 3]"
8,143255,"[[411.173828125, 121.27411651611328, 510.75317...",[0]
9,40700,"[[301.9392395019531, 135.90023040771484, 353.2...","[1, 3, 5]"


In [ ]:
sub_1.to_csv(f'/content/drive/MyDrive/SuperAI_SS4/Hackathons/SuperAI_SS4_Level_2/Hack_6_Liver_Ultrasound_Detection/submissions/confident_060.csv' , index = False)